In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/train_spam.csv')
data.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


# Анализ и предобработка данных

In [3]:
# заменим spam на 1 и ham на 0
data.loc[data['text_type'] == 'spam', 'text_type'] = 1
data.loc[data['text_type'] == 'ham', 'text_type'] = 0
data['text_type'] = data['text_type'].astype(int)

In [4]:
# посмотрим на пропуски
data.isna().sum()

text_type    0
text         0
dtype: int64

In [5]:
# посмотрим на сбаллансированность данных
data.groupby(by='text_type').count()

,text
text_type,
0,11469
1,4809


In [6]:
# предобработаем текста

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')


def clear_text(text: str, method=None):
    """
    Функция для предобработки текста

    Parameters
    ----------
    text
    method:
        - 'lemmatize': использовать WordNetLemmatizer()
        - 'stem': использовать PorterStemmer()
        - None: не использовать обработку слов

    Returns
    -------
    Текст, готовый к векторизации
    """
    stopwords_set = set(stopwords.words('english'))

    text = text.strip().lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # удаление пунктуации
    text = re.sub(r'\d+', '', text)  # удаление чисел
    text_list = text.split()
    text_list = [word for word in text_list if word not in stopwords_set]
    if method == 'lemmatize':
        lemmatizer = WordNetLemmatizer()
        text_list = [lemmatizer.lemmatize(word) for word in text_list]
    elif method == 'stem':
        stemmer = PorterStemmer()
        text_list = [stemmer.stem(word) for word in text_list]
    text = ' '.join(text_list)
    return text


data['cleared_text'] = data['text'].apply(clear_text)
data['cleared_text_stem'] = data['text'].apply(clear_text, method='stem')
data['cleared_text_lemmatize'] = data['text'].apply(clear_text, method='lemmatize')
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kudr.max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kudr.max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text_type,text,cleared_text,cleared_text_stem,cleared_text_lemmatize
0,0,make sure alex knows his birthday is over in f...,make sure alex knows birthday fifteen minutes ...,make sure alex know birthday fifteen minut far...,make sure alex know birthday fifteen minute fa...
1,0,a resume for john lavorato thanks vince i will...,resume john lavorato thanks vince get moving r...,resum john lavorato thank vinc get move right ...,resume john lavorato thanks vince get moving r...
2,1,plzz visit my website moviesgodml to get all m...,plzz visit website moviesgodml get movies free...,plzz visit websit moviesgodml get movi free al...,plzz visit website moviesgodml get movie free ...
3,1,urgent your mobile number has been awarded wit...,urgent mobile number awarded £ prize guarantee...,urgent mobil number award £ prize guarante cal...,urgent mobile number awarded £ prize guarantee...
4,0,overview of hr associates analyst project per ...,overview hr associates analyst project per dav...,overview hr associ analyst project per david r...,overview hr associate analyst project per davi...


In [7]:
X = data['cleared_text_stem']
y = data['text_type']

In [8]:
# векторизируем текст

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vect = TfidfVectorizer()
# count_vect = CountVectorizer()

tfidf_vect.fit(X_train)
# count_vect.fit(X_train)

X_train_vect = tfidf_vect.transform(X_train)
X_test_vect = tfidf_vect.transform(X_test)
# X_train_vect = count_vect.transform(X_train)
# X_test_vect = count_vect.transform(X_test)

# Обучение модели

Рассмотрим несколько моделей и для каждой из них подберем оптимальные гиперпараметры

In [9]:
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
import numpy as np

### Логистическая регрессия

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='roc_auc',
    cv=5)

grid_search.fit(X_train_vect, y_train)

print(f'Лучший roc_auc: {grid_search.best_score_}')
print(f'При параметрах: {grid_search.best_params_}')

Лучший roc_auc: 0.9776756178251041
При параметрах: {'C': 10}


In [11]:
model = grid_search.best_estimator_
model.fit(X_train_vect, y_train)

y_pred = model.predict(X_test_vect)
y_pred_proba = model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC on test: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC on test: 0.9819255484105828
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2321
           1       0.91      0.88      0.89       935

    accuracy                           0.94      3256
   macro avg       0.93      0.92      0.93      3256
weighted avg       0.94      0.94      0.94      3256



### SVM

In [13]:
from sklearn.svm import SVC

model = SVC()

parameters = {
    'C': [1, 10]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='roc_auc',
    cv=5)

grid_search.fit(X_train_vect, y_train)

print(f'Лучший roc_auc: {grid_search.best_score_}')
print(f'При параметрах: {grid_search.best_params_}')

Лучший roc_auc: 0.9807214073960498
При параметрах: {'C': 10}


In [16]:
model = grid_search.best_estimator_
model.fit(X_train_vect, y_train)

y_pred = model.predict(X_test_vect)
print(f"ROC-AUC on test: {roc_auc_score(y_test, y_pred)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC on test: 0.9242998707453683
Classification report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      2321
           1       0.94      0.87      0.90       935

    accuracy                           0.95      3256
   macro avg       0.94      0.92      0.93      3256
weighted avg       0.95      0.95      0.95      3256



### Градиентный бустинг (CatBoost)

In [24]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, classification_report

train_pool = Pool(
    data=X_train_vect,
    label=y_train,
)

test_pool = Pool(
    data=X_test_vect,
    label=y_test,
)

parameters = {
    'learning_rate': [0.03, 0.1],
    'depth': [4, 6, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
#
# catboost_model = CatBoostClassifier(
#     metric_period=50,
#     iterations=200,
#     loss_function='Logloss',
#     eval_metric='AUC',
# )
#
# catboost_model.grid_search(
#     parameters,
#     X=X_train_vect,
#     y=y_train,
#     cv=3
# )

# catboost_model = CatBoostClassifier(
#     metric_period=50,
#     use_best_model=True,
#     iterations=1100,
#     depth=7,
#     learning_rate=0.1,
#     loss_function='Logloss',
#     eval_metric='AUC',
#     random_seed=42,
# )

catboost_model = CatBoostClassifier(
    metric_period=50,
    iterations=1200,
    loss_function='Logloss',
    eval_metric='AUC',
)
catboost_model.fit(train_pool, eval_set=test_pool)

Learning rate set to 0.055181
0:	test: 0.7265730	best: 0.7265730 (0)	total: 117ms	remaining: 2m 20s
50:	test: 0.9279119	best: 0.9279119 (50)	total: 6.87s	remaining: 2m 34s
100:	test: 0.9465708	best: 0.9465708 (100)	total: 10.1s	remaining: 1m 49s
150:	test: 0.9566204	best: 0.9566204 (150)	total: 13.2s	remaining: 1m 31s
200:	test: 0.9644667	best: 0.9644667 (200)	total: 16.3s	remaining: 1m 21s
250:	test: 0.9689340	best: 0.9689340 (250)	total: 24.4s	remaining: 1m 32s
300:	test: 0.9722547	best: 0.9722547 (300)	total: 28.5s	remaining: 1m 25s
350:	test: 0.9740908	best: 0.9740908 (350)	total: 31.9s	remaining: 1m 17s
400:	test: 0.9754372	best: 0.9754372 (400)	total: 35.1s	remaining: 1m 9s
450:	test: 0.9767501	best: 0.9767501 (450)	total: 38.3s	remaining: 1m 3s
500:	test: 0.9777544	best: 0.9777544 (500)	total: 41.7s	remaining: 58.2s
550:	test: 0.9780445	best: 0.9780445 (550)	total: 44.9s	remaining: 52.9s
600:	test: 0.9789647	best: 0.9789647 (600)	total: 48.1s	remaining: 48s
650:	test: 0.9792732	

In [25]:
y_pred = catboost_model.predict(X_test_vect)
y_pred_proba = catboost_model.predict_proba(X_test_vect)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}")

ROC-AUC: 0.981934534026685
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96      2321
           1       0.94      0.83      0.88       935

    accuracy                           0.94      3256
   macro avg       0.94      0.90      0.92      3256
weighted avg       0.94      0.94      0.94      3256

